--- Day 19: Aplenty ---

The Elves of Gear Island are thankful for your help and send you on your way. They even have a hang glider that someone stole from Desert Island; since you're already going that direction, it would help them a lot if you would use it to get down there and return it to them.

As you reach the bottom of the relentless avalanche of machine parts, you discover that they're already forming a formidable heap. Don't worry, though - a group of Elves is already here organizing the parts, and they have a system.

To start, each part is rated in each of four categories:

    x: Extremely cool looking
    m: Musical (it makes a noise when you hit it)
    a: Aerodynamic
    s: Shiny

Then, each part is sent through a series of workflows that will ultimately accept or reject the part. Each workflow has a name and contains a list of rules; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)

Consider the workflow ex{x>10:one,m<20:two,a>30:R,A}. This workflow is named ex and contains four rules. If workflow ex were considering a specific part, it would perform the following steps in order:

    Rule "x>10:one": If the part's x is more than 10, send the part to the workflow named one.
    Rule "m<20:two": Otherwise, if the part's m is less than 20, send the part to the workflow named two.
    Rule "a>30:R": Otherwise, if the part's a is more than 30, the part is immediately rejected (R).
    Rule "A": Otherwise, because no other rules matched the part, the part is immediately accepted (A).

If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is accepted (sent to A) or rejected (sent to R), the part immediately stops any further processing.

The system works, but it's not keeping up with the torrent of weird metal shapes. The Elves ask if you can help sort a few parts and give you the list of workflows and some part ratings (your puzzle input). For example:

px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}

The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named in. In this example, the five listed parts go through the following workflows:

    {x=787,m=2655,a=1222,s=2876}: in -> qqz -> qs -> lnx -> A
    {x=1679,m=44,a=2067,s=496}: in -> px -> rfg -> gd -> R
    {x=2036,m=264,a=79,s=2244}: in -> qqz -> hdj -> pv -> A
    {x=2461,m=1339,a=466,s=291}: in -> px -> qkq -> crn -> R
    {x=2127,m=1623,a=2188,s=1013}: in -> px -> rfg -> A

Ultimately, three parts are accepted. Adding up the x, m, a, and s rating for each of the accepted parts gives 7540 for the part with x=787, 4623 for the part with x=2036, and 6951 for the part with x=2127. Adding all of the ratings for all of the accepted parts gives the sum total of 19114.

Sort through all of the parts you've been given; what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?

In [ ]:
file_input = open(file="./Inputs/Day19.txt", mode='r')
my_input = file_input.read()

In [ ]:
class Part:
    def __init__(self, x, m, a, s):
        self.x = x
        self.m = m
        self.a = a
        self.s = s

class Rule:
    all_rules = {}
    
    def __init__(self, name, condition, if_true, if_false):
        if '<' in condition:
            self.condition_symb = '<'
        elif '>' in condition:
            self.condition_symb = '>'
        
        self.condition_sub = condition.split(self.condition_symb)[0]
        self.condition_num = int(condition.split(self.condition_symb)[1])
        
        # Could be etither A, R or a string with the name of the rule
        # that should be checked
        self.if_true = if_true
        # This one can also be a Rule object
        self.if_false = if_false
        
        if name:
            Rule.all_rules[name] = self
    
    def __repr__(self):
        return f"Rule({self.condition_sub}{self.condition_symb}{self.condition_num}:{self.if_true}, {self.if_false})"
    
    def accepts(self, part):
        condition = None
        if self.condition_symb == '<':
            condition = lambda sub : sub < self.condition_num
        elif self.condition_symb == '>':
            condition = lambda sub : sub > self.condition_num
        
        checking_num = None
        if self.condition_sub == 'x':
            checking_num = part.x
        elif self.condition_sub == 'm':
            checking_num = part.m
        elif self.condition_sub == 'a':
            checking_num = part.a
        elif self.condition_sub == 's':
            checking_num = part.s
        
        if condition(checking_num):
            if  self.if_true == 'A':
                return True
            elif self.if_true == 'R':
                return False
            else:
                return Rule.all_rules[self.if_true].accepts(part)
        else:
            if  self.if_false == 'A':
                return True
            elif self.if_false == 'R':
                return False
            elif type(self.if_false) == Rule:
                return self.if_false.accepts(part)
            else:
                return Rule.all_rules[self.if_false].accepts(part)
Rule.all_rules.clear()

def makeRule(name, content):
    condition = content.split(":")[0]
    
    if_true = content[len(condition)+1:].split(",")[0]
    
    if_false = content[len(condition)+1 + len(if_true)+1:]
    if ':' in if_false:
        if_false = makeRule('', if_false)
    
    return Rule(name, condition, if_true, if_false)

def readRules():
    for line in my_input.splitlines():
        if not line:
            break
        
        name = line.split("{")[0]
        content = line.split("{")[1].split("}")[0]
        makeRule(name, content)

readRules()

total = 0
for line in my_input.splitlines():
    if not line or line[0] != '{':
        continue
    
    x = int(line.split("=")[1].split(",")[0])
    m = int(line.split("=")[2].split(",")[0])
    a = int(line.split("=")[3].split(",")[0])
    s = int(line.split("=")[4].split(",")[0][:-1])
    
    if Rule.all_rules["in"].accepts(Part(x, m, a, s)):
        total += x + m + a + s

print(total)

--- Part Two ---

Even with your help, the sorting process still isn't fast enough.

One of the Elves comes up with a new plan: rather than sort parts individually through all of these workflows, maybe you can figure out in advance which combinations of ratings will be accepted or rejected.

Each of the four ratings (x, m, a, s) can have an integer value ranging from a minimum of 1 to a maximum of 4000. Of all possible distinct combinations of ratings, your job is to figure out which ones will be accepted.

In the above example, there are 167409079868000 distinct combinations of ratings that will be accepted.

Consider only your list of workflows; the list of part ratings that the Elves wanted you to sort is no longer relevant. How many distinct combinations of ratings will be accepted by the Elves' workflows?

In [ ]:
class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def __repr__(self) -> str:
        return f"({self.start}, {self.end})"

    def __len__(self):
        return self.end - self.start + 1

class Parts:
    def __init__(self, x_inter : Interval, m_inter : Interval, a_inter : Interval, s_inter : Interval):
        self.x_inter : Interval = x_inter
        self.m_inter : Interval = m_inter
        self.a_inter : Interval = a_inter
        self.s_inter : Interval = s_inter
    
    def __repr__(self) -> str:
        return f"x={{{self.x_inter}}}, m={{{self.m_inter}}}, a{{{self.a_inter}}}, s={{{self.s_inter}}}"
    
    def __len__(self):
        return len(self.x_inter) * len(self.m_inter) * len(self.a_inter) * len(self.s_inter)
        

class Rule:
    all_rules = {}
    
    def __init__(self, name, condition, if_true, if_false):
        if '<' in condition:
            self.condition_symb = '<'
        elif '>' in condition:
            self.condition_symb = '>'
        
        self.condition_sub = condition.split(self.condition_symb)[0]
        self.condition_num = int(condition.split(self.condition_symb)[1])
        
        # Could be etither A, R or a string with the name of the rule
        # that should be checked
        self.if_true = if_true
        # This one can also be a Rule object
        self.if_false = if_false
        
        if name:
            Rule.all_rules[name] = self
    
    def __repr__(self):
        return f"{self.condition_sub}{self.condition_symb}{self.condition_num}:{self.if_true}, {self.if_false}"
    
    def howManyAccepts(self, parts : Parts):
        needs_to_split = None
        complies = None
        if self.condition_symb == '<':
            needs_to_split = lambda inter : inter.start < self.condition_num <= inter.end
            complies = lambda inter : inter.end < self.condition_num
        elif self.condition_symb == '>':
            needs_to_split = lambda inter : inter.start <= self.condition_num < inter.end
            complies = lambda inter : inter.start > self.condition_num
        
        checking_inter = None
        if self.condition_sub == 'x':
            checking_inter = parts.x_inter
        elif self.condition_sub == 'm':
            checking_inter = parts.m_inter
        elif self.condition_sub == 'a':
            checking_inter = parts.a_inter
        elif self.condition_sub == 's':
            checking_inter = parts.s_inter
        
        complient_parts = None
        not_complient_parts = None
        if needs_to_split(checking_inter):
            if self.condition_symb == '<':
                new_inter_low = Interval(checking_inter.start, self.condition_num-1)
                new_inter_high = Interval(self.condition_num, checking_inter.end)
            elif self.condition_symb == '>':
                new_inter_low = Interval(checking_inter.start, self.condition_num)
                new_inter_high = Interval(self.condition_num+1, checking_inter.end)
        
            if self.condition_sub == 'x':
                spplited_parts_low = Parts(new_inter_low, parts.m_inter, parts.a_inter, parts.s_inter)
                spplited_parts_high = Parts(new_inter_high, parts.m_inter, parts.a_inter, parts.s_inter)
            elif self.condition_sub == 'm':
                spplited_parts_low = Parts(parts.x_inter, new_inter_low, parts.a_inter, parts.s_inter)
                spplited_parts_high = Parts(parts.x_inter, new_inter_high, parts.a_inter, parts.s_inter)
            elif self.condition_sub == 'a':
                spplited_parts_low = Parts(parts.x_inter, parts.m_inter, new_inter_low, parts.s_inter)
                spplited_parts_high = Parts(parts.x_inter, parts.m_inter, new_inter_high, parts.s_inter)
            elif self.condition_sub == 's':
                spplited_parts_low = Parts(parts.x_inter, parts.m_inter, parts.a_inter, new_inter_low)
                spplited_parts_high = Parts(parts.x_inter, parts.m_inter, parts.a_inter, new_inter_high)
            
            if self.condition_symb == '<':
                complient_parts = spplited_parts_low
                not_complient_parts = spplited_parts_high
            elif self.condition_symb == '>':
                complient_parts =  spplited_parts_high
                not_complient_parts = spplited_parts_low
        else:
            if complies(checking_inter):
                complient_parts = parts
            else:
                not_complient_parts = parts
        
        
        total = 0
        if complient_parts:
            if self.if_true == 'A':
                total += len(complient_parts)
            elif len(self.if_true) > 1:
                total += Rule.all_rules[self.if_true].howManyAccepts(complient_parts)
        
        if not_complient_parts:
            if self.if_false == 'A':
                total += len(not_complient_parts)
            elif type(self.if_false) == Rule:
                total += self.if_false.howManyAccepts(not_complient_parts)
            elif len(self.if_false) > 1:
                total += Rule.all_rules[self.if_false].howManyAccepts(not_complient_parts)

        return total
Rule.all_rules.clear()

readRules()
inter1to4000 = Interval(1, 4000)
print(Rule.all_rules["in"].howManyAccepts(Parts(inter1to4000, inter1to4000, inter1to4000, inter1to4000)))